# Sudo Code - Fraud Detection
Team Members: Zhu Hanming, Tan Jin, Cao Wenjie, Noel Mathew Isaac

Our submission for Round 2.

In [85]:
import pandas as pd

In [86]:
## importing csvs
orders_df = pd.read_csv('orders.csv')
devices_df = pd.read_csv('devices.csv')
credit_df = pd.read_csv('credit_cards.csv')
bank_df = pd.read_csv('bank_accounts.csv')

In [87]:
## UFDS
class DisjointSet(object):

    def __init__(self):
        self.leader = {} # maps a member to the group's leader
        self.group = {} # maps a group leader to the group (which is a set)

    def add(self, a, b):
        leadera = self.leader.get(a)
        leaderb = self.leader.get(b)
        if leadera is not None:
            if leaderb is not None:
                if leadera == leaderb: return # nothing to do
                groupa = self.group[leadera]
                groupb = self.group[leaderb]
                if len(groupa) < len(groupb):
                    a, leadera, groupa, b, leaderb, groupb = b, leaderb, groupb, a, leadera, groupa
                groupa |= groupb
                del self.group[leaderb]
                for k in groupb:
                    self.leader[k] = leadera
            else:
                self.group[leadera].add(b)
                self.leader[b] = leadera
        else:
            if leaderb is not None:
                self.group[leaderb].add(a)
                self.leader[a] = leaderb
            else:
                self.leader[a] = self.leader[b] = a
                self.group[a] = set([a, b])

In [88]:
ds = DisjointSet()

In [89]:
devices_df.head()

,userid,device
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=
3,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=
4,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=


In [90]:
## Grouping the users by device ids
device_dict = {}
for index, row in devices_df.iterrows():
    userid = row['userid']
    device = row['device']
    if device not in device_dict:
        device_dict[device] = [userid]
    else:
        device_dict[device].append(userid)


In [91]:
credit_df.head()

,userid,credit_card
0,2579938,832299xxxxxx4902|02-2019
1,2154902,322429xxxxxx3848|05-2020
2,5578604,322429xxxxxx1293|11-2016
3,2051728,388329xxxxxx4303|09-2016
4,6216212,322421xxxxxx4238|05-2018


In [92]:
## Grouping the users by credit card no.
credit_dict = {}
for index, row in credit_df.iterrows():
    userid = row['userid']
    credit = row['credit_card']
    if credit not in credit_dict:
        credit_dict[credit] = [userid]
    else:
        credit_dict[credit].append(userid)

In [93]:
bank_df.head()

,userid,bank_account
0,21829134,923302000003892
1,95910542,11002023212822
2,96941876,189303223
3,23452396,2280003199803
4,12647942,32002028484803


In [94]:
## Grouping the users by bank acc no.
bank_dict = {}
for index, row in bank_df.iterrows():
    userid = row['userid']
    bank = row['bank_account']
    if bank not in bank_dict:
        bank_dict[bank] = [userid]
    else:
        bank_dict[bank].append(userid)

In [95]:
## Unioning users with same device ids
for key, value in device_dict.items():
    for i in range(len(value)):
        for x in range(i+1, len(value)):
            ds.add(value[i], value[x])

In [96]:
## Unioning users with credit card no.
for key, value in credit_dict.items():
    for i in range(len(value)):
        for x in range(i+1, len(value)):
            ds.add(value[i], value[x])

In [97]:
## Unioning users with same bank account id
for key, value in bank_dict.items():
    for i in range(len(value)):
        for x in range(i+1, len(value)):
            ds.add(value[i], value[x])

In [102]:
## Function to check if representative of each set is the same
def checker2(id1, id2):
    try:
        if ds.leader[id1]==ds.leader[id2]:
            return 1
        else:
            return 0
    except:
        return 0

In [103]:
## Checking the buyer and seller ids
orders_df['is_fraud'] = orders_df.apply(lambda x: checker2(x.buyer_userid, x.seller_userid), axis=1)

In [104]:
## Exporting file
final_df = orders_df[['orderid', 'is_fraud']]
final_df.to_csv("solution2.csv", index=False)

In [105]:
final_df.head(20)

,orderid,is_fraud
0,1953278092,0
1,1953295120,0
2,1953306402,0
3,1953314712,0
4,1953381964,0
5,1953383372,0
6,1953451192,0
7,1953453596,0
8,1953479046,0
9,1953480060,0


## The below code was written but ultimately scrapped:

In [5]:
orders_df.head()

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332


In [14]:
## Mapping users into a graph (should be adj list, not matrix. oops)
adj_matrix = {}

In [15]:
## Adding users into the adj list based on device ids
for key, value in device_dict.items():
    for i in range(len(value)):
        if value[i] not in adj_matrix:
            adj_matrix[value[i]] = []
        for x in range(len(value)):
            if x!=i:
                if value[x] not in adj_matrix[value[i]]:
                    adj_matrix[value[i]].append(value[x])


In [19]:
## Adding users into the adj list based on credit card no.s
for key, value in credit_dict.items():
    for i in range(len(value)):
        if value[i] not in adj_matrix:
            adj_matrix[value[i]] = []
        for x in range(len(value)):
            if x!=i:
                if value[x] not in adj_matrix[value[i]]:
                    adj_matrix[value[i]].append(value[x])


In [23]:
## Adding users into the adj list based on the bank acc no.
for key, value in bank_dict.items():
    for i in range(len(value)):
        if value[i] not in adj_matrix:
            adj_matrix[value[i]] = []
        for x in range(len(value)):
            if x!=i:
                if value[x] not in adj_matrix[value[i]]:
                    adj_matrix[value[i]].append(value[x])


In [53]:
adj_matrix

{10499978: [],
 54526742: [],
 33152428: [8802740, 82331972],
 8802740: [33152428, 26912434, 439880, 1661244, 103854244, 22124984, 82331972],
 55020986: [12106658, 93265192],
 100471502: [],
 61471588: [36558908],
 62907954: [],
 37491720: [],
 20756502: [],
 133160044: [],
 112533348: [],
 29345296: [],
 62003492: [],
 111158712: [],
 49136406: [],
 15655730: [],
 87708778: [],
 26860728: [],
 124665716: [],
 104362298: [],
 77295640: [],
 42916456: [],
 133432602: [],
 85362946: [],
 133440156: [],
 129669524: [27536218, 98911490, 102060526],
 27536218: [129669524, 98911490, 102060526, 61226156],
 38318486: [],
 100955932: [],
 88244914: [],
 15190214: [15180794,
  11628224,
  140230896,
  14885306,
  2126492,
  39947864,
  1995584],
 15180794: [15190214, 11628224, 140230896, 39947864, 2126492, 1995584],
 11628224: [15190214, 15180794, 2126492],
 102556856: [],
 113799880: [],
 133775686: [260456],
 260456: [133775686],
 133798444: [],
 16661284: [],
 29087736: [6917000, 17012504, 23

In [49]:
## Check if users are related. HUGE MISTAKE: This is a direct neighbour check, does not allow for kth neighbour check
def checker(id1, id2):
    if not int(id1) in adj_matrix or not int(id2) in adj_matrix:
        return 0
    elif int(id2) in adj_matrix[int(id1)] or int(id1) in adj_matrix[int(id2)]:
        return 1
    else:
        return 0

In [50]:
## Processing the df
orders_df['is_fraud'] = orders_df.apply(lambda x: checker(x.buyer_userid, x.seller_userid), axis=1)

In [51]:
## Selecting columns from df
final_df = orders_df[['orderid', 'is_fraud']]

In [52]:
## Exporting results
final_df.to_csv("solution.csv", index=False)